PLN Example
Note: we assume you are starting in your home directory - to get there type $ cd ~


From the command prompt, enter the scheme environment by typing: $ guile


Load the required modules (as you are running from the scheme environment):

In [4]:
(use-modules (ice-9 readline)) 
(activate-readline)
(add-to-load-path "/usr/local/share/opencog/scm")
(add-to-load-path ".")
(use-modules (opencog))
(use-modules (opencog query))
(use-modules (opencog exec))
(use-modules (opencog logger))

#<unspecified>

(Note you may have to change the paths to the files to suit your installation) Load the deduction rule definitions - type the following 'load' commands for both deduction-rule.scm and simple-assertions.scm

Load the default deduction rules: (load "./opencog/opencog/opencog/pln/rules/term/deduction.scm") <- this contains the 'deduction-inheritance-rule' function used later in this tutorial

(For Notebook Purposes) We first Load formulas.scm, as per the code in deduction.scm. On plane guile feel free to load the deduction.scm, as is,from the above path.

In [5]:
;; =============================================================================
;; A list of the helper functions necessary in different inference rules
;; -----------------------------------------------------------------------------
;; Index
;; - simple-deduction-strength-formula
;; - inversion-strength-formula
;;------------------------------------------------------------------------------

;; =============================================================================
;; Simple Deduction Formula
;;
;; Preconditions:
;;
;;   1. Check that P(B|A) makes sense
;;
;;      1.a. P(B|A) is defined
;;
;;         0.0 < sA
;;
;;      1.b. P(B|A) is greater than or equal to P(A,B)/P(A) considering
;;      the smallest possible intersection between A and B
;;
;;         max((sA+sB-1)/sA, 0) <= sAB
;;
;;      1.c. P(B|A) is smaller than P(A,B)/P(A) considering the largest
;;      possible intersection between A and B
;;
;;         sAB <= min(1, sB/sA)
;;
;;   2. Check that P(C|B) makes sense
;;
;;      1.a. P(C|B) is defined
;;
;;         0.0 < sB
;;
;;      1.b. P(C|B) is greater than P(B,C)/P(B) considering the
;;      smallest possible intersection between B and C
;;
;;         max((sB+sC-1)/sB, 0) <= sBC
;;
;;      1.c. P(C|B) is greater than P(B,C)/P(B) considering the
;;      largest possible intersection between B and C
;;
;;         sBC <= min(1, sC/sB)
;;
;; Calculation:
;;
;;   sAC = [sAB.sBC + ((1-sAB)(sC - sBsBC))/(1-sB)]
;;
;; To avoid division by 0 (which the preconditions do not necessarily
;; prevent), i.e. when sB tends to 1 we find that
;;
;;   sAC tends to sC
;;
;; Indeed, in such case sAB tends to sB and sBC tends to sC. As a
;; result (according to the main formula above) sAC tends to
;;
;;   sB.sC + ((1-sB)(sC - sBsC))/(1-sB) = sB.sC + (sC - sB.sC) = sC
;;
;; If the preconditions are not met it returns sAC = 0, although
;; honnestly the deduction rule should not have been applied in the
;; first place.
;; -----------------------------------------------------------------------------


;; Limit an number to be within a certain range
(define (limit x l u)
  (max l (min u x)))

; Consistency Conditions
(define (smallest-intersection-probability sA sB)
  (limit (/ (+ sA sB -1) sA) 0 1))

(define (largest-intersection-probability sA sB)
  (limit (/ sB sA) 0 1))

(define (conditional-probability-consistency sA sB sAB)
  (and (< 0 sA)
       (<= (smallest-intersection-probability sA sB) sAB)
       (<= sAB (largest-intersection-probability sA sB))))

; Main Formula

(define (simple-deduction-strength-formula sA sB sC sAB sBC)
  (if
     (and
        (conditional-probability-consistency sA sB sAB)
        (conditional-probability-consistency sB sC sBC))
     ;; Preconditions are met
     (if (< 0.99 sB)
        ;; sB tends to 1
        sC
        ;; otherwise
        (+ (* sAB sBC) (/ (* (- 1 sAB) (- sC (* sB sBC))) (- 1 sB))))
     ;; Preconditions are not met
     0))

;; =============================================================================
;; Inversion Formula
;;
;; Preconditions:
;;
;;   1. Check that P(B|A) makes sense
;;
;;      1.a. P(B|A) is defined
;;
;;         0.0 < sA
;;
;;      1.b. P(B|A) is greater than or equal to P(A,B)/P(A) considering
;;      the smallest possible intersection between A and B
;;
;;         max((sA+sB-1)/sA, 0) <= sAB
;;
;;      1.c. P(B|A) is smaller than P(A,B)/P(A) considering the largest
;;      possible intersection between A and B
;;
;;         sAB <= min(1, sB/sA)
;;
;;   2. Check that P(A|B) makes sense
;;
;;      1. P(A|B) is defined
;;
;;         0.0 < sB
;;
;;         And that is all, ultimately the calculation should result
;;         into a consistent sBA (or let's hope so).
;;
;; Calculation:
;;
;;   sBA = (sAB * sB) / sA
;;
;; -----------------------------------------------------------------------------

(define (inversion-consistency sA sB sAB)
  (and (< 0 sA)
       (< 0 sB)
       (<= (smallest-intersection-probability sA sB) sAB)
       (<= sAB (largest-intersection-probability sA sB))))

(define (inversion-strength-formula sA sB sAB)
  (if (inversion-consistency sA sB sAB)
      (/ (* sAB sA) sB)
      0))

;; =============================================================================
;; Invert formula
;;
;; Inverts the input
;; -----------------------------------------------------------------------------

(define (invert a)
    (/ 1.0 a))

;; =============================================================================
;; Negate formula
;;
;; Negates the probability
;; -----------------------------------------------------------------------------

(define (negate a)
    (- 1 a))

;; =============================================================================
;; Transitive Similarity Formula
;;
;; Returns the strength value of the transitive similarity rule
;; -----------------------------------------------------------------------------

(define (transitive-similarity-strength-formula sA sB sC sAB sBC )
    (let
        ((T1 (/ (* (+ 1 (/ sB sA)) (sAB)) (+ 1 sAB)))
         (T2 (/ (* (+ 1 (/ sC sB)) (sBC)) (+ 1 sBC)))
         (T3 (/ (* (+ 1 (/ sB sC)) (sBC)) (+ 1 sBC)))
         (T4 (/ (* (+ 1 (/ sA sB)) (sAB)) (+ 1 sAB))))
        (invert (- (+ (invert (+ (* T1 T2) (* (negate T1) (/ (- sC (* sB T2)) (negate sB)))))
                      (invert (+ (* T3 T4) (* (negate T3) (/ (- sC (* sB T4)) (negate sB)))))) 1))))

;; =============================================================================
;; PreciseModusPonens formula
;;
;; Returns the strength value of the precise modus ponens rule
;; -----------------------------------------------------------------------------

(define (precise-modus-ponens-strength-formula sA sAB snotAB)
    (+ (* sAB sA) (* snotAB (negate sA))))


#<unspecified>

In [6]:
;; =============================================================================
;; DeductionRule
;;
;; <LinkType>
;;   A
;;   B
;; <LinkType>
;;   B
;;   C
;; |-
;; <LinkType>
;;   A
;;   C
;;
;; Due to type system limitations, the rule has been divided into 3:
;;       deduction-inheritance-rule
;;       deduction-implication-rule
;;       deduction-subset-rule
;;
;; This deduction rule makes assumptions to avoid having too many
;; premises. Another more precise rule should be created as well.
;;
;; -----------------------------------------------------------------------------

;; Generate the corresponding deduction rule given its link-type and
;; the type for each variable (the same for all 3).
(define (gen-deduction-rule link-type var-type)
  (let* ((A (Variable "$A"))
         (B (Variable "$B"))
         (C (Variable "$C"))
         (AB (link-type A B))
         (BC (link-type B C))
         (AC (link-type A C)))
    (Bind
      (VariableList
        (TypedVariable A var-type)
        (TypedVariable B var-type)
        (TypedVariable C var-type))
      (And
        AB
        BC
        (Not (Identical A C)))
      (ExecutionOutput
        (GroundedSchema "scm: deduction-formula")
        (List
          ;; Conclusion
          AC
          ;; Premises
          ;;
          ;; TODO: perhaps A, B, C should be added as premises as
          ;; they are used in the formula.
          AB
          BC)))))

(define deduction-inheritance-rule
  (let ((var-type (TypeChoice
                    (TypeNode "ConceptNode")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule InheritanceLink var-type)))

(define deduction-implication-rule
  (let ((var-type (TypeChoice
                    (TypeNode "PredicateNode")
                    (TypeNode "LambdaLink")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule ImplicationLink var-type)))

(define deduction-subset-rule
  (let ((var-type (TypeChoice
                    (TypeNode "ConceptNode")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule SubsetLink var-type)))

(define (deduction-formula conclusion . premises)
  (if (= (length premises) 2)
    (let*
        ((AC conclusion)
         (AB (list-ref premises 0))
         (BC (list-ref premises 1))
         (sA (cog-stv-strength (gar AB)))
         (cA (cog-stv-confidence (gar AB)))
         (sB (cog-stv-strength (gar BC)))
         (cB (cog-stv-confidence (gar BC)))
         (sC (cog-stv-strength (gdr BC)))
         (cC (cog-stv-confidence (gdr BC)))
         (sAB (cog-stv-strength AB))
         (cAB (cog-stv-confidence AB))
         (sBC (cog-stv-strength BC))
         (cBC (cog-stv-confidence BC))
         (alpha 0.9) ; how much confidence is lost at each deduction step

         ;; Hacks to overcome the lack of distributional TV. If s=1
         ;; and c=0, then assign s to the mode value satisfying the
         ;; deduction consistency constraint (what a pain, let's use
         ;; 0.25 for now).
         (sA (if (and (< 0.99 sA) (<= cA 0)) 0.25 sA))
         (sB (if (and (< 0.99 sB) (<= cB 0)) 0.25 sB))
         (sC (if (and (< 0.99 sC) (<= cC 0)) 0.25 sC)))
      (if (and
           (or (= 0 cA) (= 0 cB) (= 0 cAB)
               (conditional-probability-consistency sA sB sAB))
           (or (= 0 cB) (= 0 cC) (= 0 cBC)
               (conditional-probability-consistency sB sC sBC)))
          (if (< 0.99 (* sB cB))
              ;; Hack to overcome for the lack of distributional
              ;; TV. This covers the case where B fully confidently
              ;; tends to 1. See formulas.scm Simple Deduction
              ;; Formula comment for more explanations. This
              ;; overlaps with the implication-introduction-rule.
              (let ((sAC sC)
                    (cAC (* alpha cA cC)))
                (if (and (< 1e-8 sAC) (< 1e-8 cAC)) ;; Don't create zero
                                              ;; knowledge. Note that
                                              ;; sAC == 0 is not zero
                                              ;; knowledge but it's
                                              ;; annoying in the
                                              ;; current hacky
                                              ;; situation.
                    (cog-merge-hi-conf-tv! AC (stv sAC cAC))))
              ;; Branch if sB * cB <= 0.99
              (let* ((sAC (if (or (< 0.99 (* sAB cAB)) (< 0.99 (* sBC cBC)))
                              ;; Hack to overcome for the lack of
                              ;; distributional TV. This covers the case
                              ;; where little is known about A and B
                              ;; (i.e. their strength is meaningless), yet
                              ;; we can confidently calculate sAC because
                              ;; sAB and sBC are so high anyway.
                              (* sAB sBC)
                              ;; Otherwise fall back on the naive formula
                              (simple-deduction-strength-formula sA sB sC sAB sBC)))
                     (cAC (min cAB cBC))
                     ;; Unless the 2 implication are fully confident
                     ;; decrease the confidence by some factor. I'm not
                     ;; sure how justify this for now, it's perhaps a
                     ;; bad hack.
                     (cAC (* (if (< cAC 0.99) alpha 1.0) cAC)))
                (if (and (< 1e-8 sAC) (< 1e-8 cAC)) ;; Don't create zero
                                              ;; knowledge. Note that
                                              ;; sAC == 0 is not zero
                                              ;; knowledge but it's
                                              ;; annoying in the
                                              ;; current hacky
                                              ;; situation.
                    (cog-merge-hi-conf-tv! AC (stv sAC cAC)))))))))

;; Name the rules
(define deduction-inheritance-rule-name
  (DefinedSchemaNode "deduction-inheritance-rule"))
(DefineLink deduction-inheritance-rule-name
  deduction-inheritance-rule)

(define deduction-implication-rule-name
  (DefinedSchemaNode "deduction-implication-rule"))
(DefineLink deduction-implication-rule-name
  deduction-implication-rule)

(define deduction-subset-rule-name
  (DefinedSchemaNode "deduction-subset-rule"))


#<unspecified>

In [7]:
(DefineLink deduction-subset-rule-name
  deduction-subset-rule)

(DefineLink
   (DefinedSchemaNode "deduction-subset-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$A")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$B")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$C")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
      )
      (AndLink
         (SubsetLink
            (VariableNode "$B")
            (VariableNode "$C")
         )
         (SubsetLink


# (DefineLink
   (DefinedSchemaNode "deduction-subset-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$A")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$B")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$C")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
      )
      (AndLink
         (SubsetLink
            (VariableNode "$B")
            (VariableNode "$C")
         )
         (SubsetLink
            (VariableNode "$A")
            (VariableNode "$B")
         )
         (NotLink
            (IdenticalLink
               (VariableNode "$A")
               (VariableNode "$C")
            )
         )
      )
      (ExecutionOutputLink
         (GroundedSchemaNode "scm: deduction-formula")
         (ListLink
            (SubsetLink
               (VariableNode "$A")
               (VariableNode "$C")
            )
            (SubsetLink
               (VariableNode "$A")
               (VariableNode "$B")
            )
            (SubsetLink
               (VariableNode "$B")
               (VariableNode "$C")
            )
         )
      )
   )
)

Load this file containing custom functions to query the atomspace:(load "./opencog/tests/pln/rules/simple-assertions.scm")<- this contains the 'find-humans' function used later in this tutorial

In [8]:
; Define the concepts

(ConceptNode "Socrates" (stv .001 0.9))
(ConceptNode "Einstein" (stv .001 0.9))
(ConceptNode "Peirce" (stv .001 0.9))
(ConceptNode "man" (stv .01 0.9))
(ConceptNode "human" (stv .02 0.9))

; Define the instances of man

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Socrates")
    (ConceptNode "man"))

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Einstein")
    (ConceptNode "man"))

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Peirce")
    (ConceptNode "man"))

; Define what man is part of

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "man")
    (ConceptNode "human"))

; Assign some additional memberships as well

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Einstein")
    (ConceptNode "violin-players"))

; Pattern to match to check the output

(define find-humans
    (BindLink
        (VariableNode "$X")
        (InheritanceLink
            (VariableNode "$X")
            (ConceptNode "human"))
        (ListLink
            (VariableNode "$X"))))

#<unspecified>

The command 'cog-execute! find-humans' queries all humans - go ahead and run it from the scheme (guile) prompt:

In [9]:
(cog-execute! find-humans)

(SetLink
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
)


Observe that there is only one instance of human defined. Output should be:

$6 = (SetLink
   (ListLink
      (ConceptNode "man" (stv 0.0099999998 0.89999998))
   )
)

The command 'cog-bind deduction-inheritance-rule' creates new relationships - deduces new facts about relationships. It will find relationships between various existing nodes (Einstein, Socrates, Peirce) to the node 'human'. Go ahead, run the command it:

In [10]:
 (cog-execute! deduction-inheritance-rule)

(SetLink
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Einstein" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Socrates" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Peirce" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
)


You can see from the output that it deduced that there were three other humans

$6 = (SetLink
   (InheritanceLink (stv 0.81111109 0.81)
      (ConceptNode "Einstein" (stv 0.001 0.89999998))
      (ConceptNode "human" (stv 0.02 0.89999998))
   )
   (InheritanceLink (stv 0.81111109 0.81)
      (ConceptNode "Socrates" (stv 0.001 0.89999998))
      (ConceptNode "human" (stv 0.02 0.89999998))
   )
   (InheritanceLink (stv 0.81111109 0.81)
      (ConceptNode "Peirce" (stv 0.001 0.89999998))
      (ConceptNode "human" (stv 0.02 0.89999998))
   )
)

Run (cog-execute! find-humans) command again. Now you should see more humans. Observe that there are 3 additional instances of human defined (Socrates, Einstein & Peirce):

In [11]:
(cog-execute! find-humans)

(SetLink
   (ListLink
      (ConceptNode "Socrates" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "Einstein" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
   (ListLink
      (ConceptNode "Peirce" (stv 0.001 0.9))
   )
)


$7 = (SetLink
   (ListLink
      (ConceptNode "Socrates" (stv 0.001 0.89999998))
   )
   (ListLink
      (ConceptNode "Einstein" (stv 0.001 0.89999998))
   )
   (ListLink
      (ConceptNode "man" (stv 0.0099999998 0.89999998))
   )
   (ListLink
      (ConceptNode "Peirce" (stv 0.001 0.89999998))
   )
)

Create new concept nodes yourself and make a new inheritance link

You can create new ConceptNodes yourself with inheritance from human. For example type:
Create a new ConceptNode 'Adam Ford': (ConceptNode "Adam Ford" (stv .001 0.9)) Note: stv stands for SimpleTruthValue
Create an inheritance link to man: (Inheritance (stv .9 .9) (ConceptNode "Adam Ford") (ConceptNode "man")) Note, the PLN deduction rule is probabilistic and will check whether the stvs on the concept nodes and inheritance links are consistent with probability theory, will silently discard inferences if they're not, see [1] for more details. Likewise the confidence (second value of the stv should be strictly positive, otherwise the deduction will likely not be able to infer anything useful.
Try to find-humans (before running '(cog-bind deduction-inheritance-rule)' to deduce inheritances): (cog-bind find-humans) the new ConceptNode shouldn't be displayed yet.
Next deduce the inheritances : (cog-bind deduction-inheritance-rule)
Now find the humans again : (cog-bind find-humans) Now 'Adam Ford' should be listed as one of the humans.
== Did you notice above that we set up an inheritance link such that 'Adam Ford' inherits from 'man'? Why didn't we just inherit from human?
Create a new concept node 'Ben Goerzel' the same way as we did above with 'Adam Ford'. Then create an inheritance link to 'human': (Inheritance (stv .9 .9) (ConceptNode "Ben Goerzel") (ConceptNode "human"))
Next deduce the inheritances (as above). What do you expect the output of (cog-bind deduction-inheritance-rule) will be?
Does "Ben Goertzel" appear...? why (or why not)? :-)

More PLN examples
See github, instructions in markdown
A deduction-rule example is given here: This link no longer exists:
https://github.com/opencog/opencog/tree/master/opencog/reasoning/pln/rules
Is this the correct link?
https://github.com/opencog/opencog/tree/master/opencog/pln/rules
Old Python 'Smokes' example: https://github.com/opencog/opencog/tree/bf4581f5a59065d0aba0052c8fbab0599990bb60/opencog/python/pln_old/examples/tuffy/smokes
Add an example stepping through some PLN inferences by hand in the Scheme shell, applying each rule by hand instead of using a chainer. The smokes example may be good for this.

PLN rules on git: https://github.com/opencog/opencog/tree/master/opencog/pln/rules